In [1]:
# |default_exp chroma_db

## Install dependencies

## Make an app with Gradio

In [2]:
# |export
import csv
import re
import chromadb
from chromadb import Settings
import pandas as pd
from dotenv import load_dotenv
import os

import gradio as gr
# from fastcore.net import urljson, HTTPError
# from openai import api_key
# from openai import OpenAI

In [3]:
# |export
# import chromadb.utils.embedding_functions as embedding_functions
# from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [4]:
# load_dotenv()
# gemini_key = os.getenv('GEMINI_API_KEY')
# deepseek_key = os.getenv('DEEPSEEK_R1_API_KEY')

In [5]:
print(os.environ.get('HTTPS_PROXY'))


None


In [6]:
# client = OpenAI(
#   base_url="https://openrouter.ai/api/v1",
#   api_key=deepseek_key,
# )

In [7]:
# deepseek_ef = OpenAIEmbeddingFunction(
#     api_key=deepseek_key,
#     api_base="https://openrouter.ai/api/v1",
#     model_name="deepseek_r1：free",
# )

In [8]:

# completion = client.chat.completions.create(
#   extra_headers={
#     "HTTP-Referer": "binjian.github.io", # Optional. Site URL for rankings on openrouter.ai.
#     "X-Title": "binjian's digital garden", # Optional. Site title for rankings on openrouter.ai.
#   },
#   extra_body={},
#   model="deepseek/deepseek-r1:free",
#   messages=[
#     {
#       "role": "user",
#       "content": "What's Anthropic's/meb Model context protocol?"
#     }
#   ]
# )
# print(completion.choices[0].message.content)

Anthropic develops AI systems like Claude, focused on being helpful, honest, and harmless. While specific internal protocols for model context aren't fully public, key principles align with industry standards for ethical AI:

1. **Data Handling**: Training on publicly available data, avoiding private or sensitive information unless properly licensed.
2. **Privacy**: Safeguarding user inputs, not storing personal data without consent.
3. **Safety**: Implementing filters and controls to prevent harmful outputs.
4. **Transparency**: Clearly stating capabilities, limitations, and that responses are AI-generated.

For details, refer to Anthropic’s official documentation or privacy policy.


In [9]:
# google_ef = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=gemini_key)

In [10]:
def convert_qa_to_csv(input_file, output_file):
    """
    Convert a text file with Q/A format to a CSV file.

    Args:
        input_file: Path to the input text file
        output_file: Path to the output CSV file
    """
    # Read the content of the file
    with open(input_file, 'r', encoding='utf-8') as f:
        content = f.read()

    # Split the content by 'Q' marker
    qa_blocks = content.split('Q\n')

    qa_blks = [block.strip() for block in qa_blocks][1:]
    # Remove empty blocks (like the first one if file starts with 'Q')
    # qa_blocks = [[line for line in block.split('\n') ] for block in qa_blks if block.strip()]
    # Remove empty blocks (like the first one if file starts with 'Q')
    # qa_blocks = [blk for block in qa_blocks if block.strip() for blk in block.strip()]

    # Process each Q&A block
    qa_pairs = []
    for block in qa_blks:
        # Split the block into lines
        lines = block.strip().split('\n')

        if lines:
            # First line is the question
            question = lines[0]
            # The rest are the answer
            answer = '\n'.join(lines[1:])

            # Add the pair to our list
            qa_pairs.append([question, answer])

    # Write to CSV
    with open(output_file, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        # Write header
        writer.writerow(['Question', 'Answer'])
        # Write Q&A pairs
        for pair in qa_pairs:
            writer.writerow(pair)
    print(f"Conversion complete. CSV file saved to {output_file}")

In [11]:

input_files = ["../res/qa_service.txt", "../res/qa_technology.txt"]
output_files = ["../res/qa_service.csv", "../res/qa_technology.csv"]
for in_f, ot_f in zip(input_files, output_files):
    convert_qa_to_csv(in_f, ot_f)
    

Conversion complete. CSV file saved to ../res/qa_service.csv
Conversion complete. CSV file saved to ../res/qa_technology.csv


In [8]:
# |export
client = chromadb.PersistentClient(path="../db")
collections = [client.get_or_create_collection(name="siasun_qa_service"),
               client.get_or_create_collection(name="siasun_qa_technology")]
# collections = [client.get_or_create_collection(name="siasun_qa_service",embedding_function=deepseek_ef),
#                 client.get_or_create_collection(name="siasun_qa_technology",embedding_function=deepseek_ef)]
# collections = [client.get_or_create_collection(name="siasun_qa_service", embedding_function=google_ef),
#                client.get_or_create_collection(name="siasun_qa_technology", embedding_function=google_ef)]

In [13]:
# |export
for csv_file,collection in zip(output_files,collections):
    with open(csv_file, newline='') as f:
        reader = csv.reader(f)
        for i, row in enumerate(reader):
            collection.add(
                documents = row,
                metadatas = [{"source": "question"}, {"source": "answer"}],
                ids = [f"{collection.name}_q{i}", f"{collection.name}_a{i}"]
            )

Add of existing embedding ID: siasun_qa_service_q0
Add of existing embedding ID: siasun_qa_service_a0
Insert of existing embedding ID: siasun_qa_service_q0
Insert of existing embedding ID: siasun_qa_service_a0
Add of existing embedding ID: siasun_qa_service_q1
Add of existing embedding ID: siasun_qa_service_a1
Insert of existing embedding ID: siasun_qa_service_q1
Insert of existing embedding ID: siasun_qa_service_a1
Add of existing embedding ID: siasun_qa_service_q2
Add of existing embedding ID: siasun_qa_service_a2
Insert of existing embedding ID: siasun_qa_service_q2
Insert of existing embedding ID: siasun_qa_service_a2
Add of existing embedding ID: siasun_qa_service_q3
Add of existing embedding ID: siasun_qa_service_a3
Insert of existing embedding ID: siasun_qa_service_q3
Insert of existing embedding ID: siasun_qa_service_a3
Add of existing embedding ID: siasun_qa_service_q4
Add of existing embedding ID: siasun_qa_service_a4
Insert of existing embedding ID: siasun_qa_service_q4
Inse

In [6]:
#|export
queries=["你们的产品需要多久维护一次?","我怎么设置机器人的安全工作区域?"]
queries[0]

'你们的产品需要多久维护一次?'

In [9]:
results = collections[0].query(
    query_texts=queries,
    n_results=4
)

Add of existing embedding ID: siasun_qa_service_q0
Add of existing embedding ID: siasun_qa_service_a0
Add of existing embedding ID: siasun_qa_service_q1
Add of existing embedding ID: siasun_qa_service_a1
Add of existing embedding ID: siasun_qa_service_q2
Add of existing embedding ID: siasun_qa_service_a2
Add of existing embedding ID: siasun_qa_service_q3
Add of existing embedding ID: siasun_qa_service_a3
Add of existing embedding ID: siasun_qa_service_q4
Add of existing embedding ID: siasun_qa_service_a4
Add of existing embedding ID: siasun_qa_service_q5
Add of existing embedding ID: siasun_qa_service_a5
Add of existing embedding ID: siasun_qa_service_q6
Add of existing embedding ID: siasun_qa_service_a6
Add of existing embedding ID: siasun_qa_service_q7
Add of existing embedding ID: siasun_qa_service_a7
Add of existing embedding ID: siasun_qa_service_q8
Add of existing embedding ID: siasun_qa_service_a8
Add of existing embedding ID: siasun_qa_service_q9
Add of existing embedding ID: s

In [10]:
results

{'ids': [['siasun_qa_service_q2',
   'siasun_qa_service_a9',
   'siasun_qa_service_q21',
   'siasun_qa_service_a10'],
  ['siasun_qa_service_q1',
   'siasun_qa_service_a19',
   'siasun_qa_service_q22',
   'siasun_qa_service_a18']],
 'embeddings': None,
 'documents': [['你们的产品，多久需要维护一次？维护保养内容有哪些？',
   '作业作为一个重要的单元，所以不能进行批量删除与添加，防止误操作造成损失。',
   '宏作业是干什么用的？',
   '新松有标准的视觉通讯协议，视觉厂家可以按照此协议进行开发适配。当前适配过的品牌有，沈阳自动化所，欧姆龙、海康、梅卡曼德、视比特、基恩士。'],
  ['我们的人员调试不熟练，进度慢，你们厂家能负责调试吗？',
   '机器人打精度用到的开关。',
   '机器人可以存储多少个作业？',
   '零位设定，是机器人在零位时的码盘值，码盘输入、码盘输出是以零位时为零开始计数的码盘值。']],
 'uris': None,
 'data': None,
 'metadatas': [[{'source': 'question'},
   {'source': 'answer'},
   {'source': 'question'},
   {'source': 'answer'}],
  [{'source': 'question'},
   {'source': 'answer'},
   {'source': 'question'},
   {'source': 'answer'}]],
 'distances': [[0.35835238473584624,
   0.8026765812953612,
   0.8398664268040978,
   0.8905950951966901],
  [0.5583867931983068,
   0.5964258746641342,
   0.6352793698255077,
   0.73287700

In [17]:
# results['metadatas'][0] #[0]['source']
results['documents'][0][0]

'你们的产品，多久需要维护一次？维护保养内容有哪些？'

In [12]:
colls = client.list_collections()
colls

['siasun_qa_technology', 'deepseek_r1_retrieval', 'siasun_qa_service']

In [13]:
queries1 = ['你们的产品需要多久维护一次?','宏作业有什么用?']

In [14]:
def qa(questions:list[str], collections:list[chromadb.Collection]=collections):
    # Check the encoding of input question to select appropriate embedding model

    matched_questions = []
    for collection in collections:
        results = collection.query(
            query_texts=questions,
            n_results=4
        )
        docs = []
        for i,metadata in enumerate(results['metadatas'][0]):
            if metadata['source'] == 'question':
                docs.append({'id': results['ids'][0][i],
                                'document': results['documents'][0][i], 
                                'distance':results['distances'][0][i]})
        df = pd.DataFrame(docs)
        matched_questions.append(df)

    df_matched_questions = pd.concat(matched_questions,axis=0,ignore_index=True)
    best_match_q_id = df_matched_questions.loc[df_matched_questions['distance'].idxmin()]['id']
    id_a_list = best_match_q_id.split('_')
    id_a_list[-1] = id_a_list[-1].replace('q','a')
    id_a = '_'.join(id_a_list)
    coll_idx = 0 if id_a_list[-2] == 'service' else 1
    best_answer = collections[coll_idx].get(id_a)['documents']
    res_text = best_answer[0]
    return res_text
    # question =
    # return answers

In [15]:

question = "你们的产品需要多久维护一次?"
res = qa(question)
res


Add of existing embedding ID: siasun_qa_technology_q0
Add of existing embedding ID: siasun_qa_technology_a0
Add of existing embedding ID: siasun_qa_technology_q1
Add of existing embedding ID: siasun_qa_technology_a1
Add of existing embedding ID: siasun_qa_technology_q2
Add of existing embedding ID: siasun_qa_technology_a2
Add of existing embedding ID: siasun_qa_technology_q3
Add of existing embedding ID: siasun_qa_technology_a3
Add of existing embedding ID: siasun_qa_technology_q4
Add of existing embedding ID: siasun_qa_technology_a4
Add of existing embedding ID: siasun_qa_technology_q5
Add of existing embedding ID: siasun_qa_technology_a5
Add of existing embedding ID: siasun_qa_technology_q6
Add of existing embedding ID: siasun_qa_technology_a6
Add of existing embedding ID: siasun_qa_technology_q7
Add of existing embedding ID: siasun_qa_technology_a7
Add of existing embedding ID: siasun_qa_technology_q8
Add of existing embedding ID: siasun_qa_technology_a8
Add of existing embedding ID

'根据机器人的型号和实际使用情况，制定机器人的保养计划,一般分为日常、3 个月、6 个月、1 年期的维护保养。\n需要对机器人进行日常点检和定期维护保养，点检工作主要检查设备是否存在漏油、异响、异常震动、异常报警；定期维护保养主要对油脂、线束护套、风扇、电机接头等易损位置进行检查，并定期更换润滑油。具体检验项目及维护周期详见安装维护手册。'

In [16]:
print(res)

根据机器人的型号和实际使用情况，制定机器人的保养计划,一般分为日常、3 个月、6 个月、1 年期的维护保养。
需要对机器人进行日常点检和定期维护保养，点检工作主要检查设备是否存在漏油、异响、异常震动、异常报警；定期维护保养主要对油脂、线束护套、风扇、电机接头等易损位置进行检查，并定期更换润滑油。具体检验项目及维护周期详见安装维护手册。


In [19]:
# |export
iface = gr.Interface(fn=qa, inputs=gr.Text(value="多久维护一次产品?"), outputs="text")
iface.launch(width=500,share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://2205b653796d34e6b5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [20]:
# this is only necessary in a notebook
iface.close()

Closing server running on port: 7860


## Create a `requirements.txt` file

In [26]:
%%writefile ../requirements.txt
fastcore

Overwriting ../requirements.txt


In [27]:
# | hide
import nbdev

nbdev.nbdev_export()

In [28]:
# |default_exp data_preprocessing

## Convert this notebook into a Gradio app

In [29]:
# from nbdev.export import nb_export
# nb_export('01_gradio.ipynb', lib_path='.', name='gradio')

In [30]:
# | hide
import nbdev

nbdev.nbdev_export()